<a href="https://colab.research.google.com/github/zuzannazak/Tangles/blob/main/Tangle_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#To do


*   Wizualizacja i obcinanie drzewa
*   Jakiś dataset dla porównania (praca z klasyfikacją, podobny dataset)



#Tangles theoretical intro

##Definitions

Traditional clustering identifies groups of objects that share
certain qualities. Tangles do the converse: they identify
groups of qualities that often occur together. They can
thereby identify and discover types: of behaviour, views,
abilities, dispositions.



*   (potential) features s of (elements v of) V (aka separations)
   *    The elements of S are called potential features of (the elements of) V, those of S~ are their features. An element of V has a feature A if it lies in A.
*   specifications →s , ←s of such s
   *   A subset of S~ is a specification of S if for every s ∈ S it contains either →s or ←s but not both. The elements of S are called potential features of (the elements of) V, those of S~ are their features. An element of V has a feature A if it lies in A.
   *   types (or typical specifications of S) 
      *   A subset of S~ is a specification of S if for every s ∈ S it contains either →s or ←s but not both.
   *   consistency of specifications
     *   A subset of S~ is inconsistent if it contains three features →r, →s, →t such that no element of V has them al
*   tangles
   *   Every Fn-tangle of S contains all the features ←s for which →s has fewer than n elements, because it cannot contain {→s } ∈ Fn as a subset.
*   star
   *   A set of features is a star if their inverses are disjoint subsets of V. 
*   universe of features
   *   We shall be particularly interested in sets S such that S~ contains, for any given →r ,→s ∈ S~, also the features →r ∨ →s and →r ∧ →s . If S has this property, then S~ will, in fact, contain all Boolean expressions of features it contains already - combinations of features built from others by using the symbols ∨ and ∧ and taking inverses. We then call S~ a universe of features.
*   submodularity
   *   S and S~ are submodular if for any two features →r ,→s ∈ S~ at least one of →r ∨ →s and →r ∧ →s is also in S~.



#Tangles real life possible examples
https://www.lib.ncsu.edu/teaching-and-learning-datasets


*   Binary survey data from the Austrian National Guest Survey conducted in the summer season of 1997 - https://link.springer.com/chapter/10.1007/3-540-44668-0_16 
*   Learning style preference - https://data.mendeley.com/datasets/mtvfdwm3dt/1
*   Attachment styles in dogs and their relationship with separation-related disorder - https://www.sciencedirect.com/science/article/abs/pii/S0168159118304921 - https://data.mendeley.com/datasets/xncmdh494n/1 



#Tangles blueprint code - to be adjusted for chosen dataset
This section contains standarized code for tangle search tree. The code itself won't work as it should be adjusted for a specific dataset where specified. This section should also work as a documentation.

##Imports
This implementation of tangle search tree uses anytree python library.


In [ ]:
pip install openpyxl==3.0.0

In [ ]:
pip install xlrd==1.2.0

In [ ]:
pip install anytree

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np
import openpyxl
from anytree import AnyNode, Node, RenderTree, find, PostOrderIter, PreOrderIter, find_by_attr

##Data preparation
The input should be formatted as an excel file, where each row contain a separate object that is them grouped to form a tangle.

The index column is then added to the file to differenciate between objects. If there's already a column that can act as a differentiator between objects (for example name, given that its unique), this step can be omitted.

In [ ]:
df_original = pd.read_excel("URL TO THE FILE")
df_original = df_original.reset_index() #optional
df_original.head()


In case some columns should be ommited during tree creation, this is the place where its titles can be listed.

In [ ]:
df_original = df_original.drop(['column name 1', 'column name 2'], axis=1)

A new dataframe that will be amended for tangles is created. It will use the index or custom differentiator column to distinguish objects.

The code below turns the columns into binary columns containing yes or no answer for each variants in each column. If the dataset of choice already has this format, this step can be ommitted. Please keep in mind that for columns with multiple variants this step may cause creation of many columns affecting performance of tangle search tree creation.

In [ ]:
object_identifier_column_name = 'index' #replace index with the differentiator column if index step was omitted earlier

df_amended_for_tangles = pd.DataFrame()
df_amended_for_tangles[object_identifier_column_name]=df_original[object_identifier_column_name] 

for columnName in df_original.iteritems():
  if columnName[0] == object_identifier_column_name:
    print("this is the ID column")
  else:
    i=0
    original_column = columnName[0]
    arr = df_original[str(original_column)].to_numpy()
    unique_arr = np.unique(arr)
    while i < len(unique_arr):
      thiscolumnname = original_column + " - " + str(unique_arr[i])
      df_amended_for_tangles[thiscolumnname] = (np.where(df_original[original_column].values == i, "1", "0"))
      i=i+1

For testing purposes it may be easier to work on a smaller dataset, therefore if needed the frac argument in the code below can be adjusted to sample the % of the dataset that will be used in tangle search tree creation.

In [ ]:
df_amended_for_tangles = df_amended_for_tangles.sample(frac=0.05, replace=False, random_state=1)
df_amended_for_tangles

##Tangle search tree creation

This part of the code calculates the cost function for all the questions/columns in the dataset. The cost function is a sum of similarities between objects divided by that question, normalized by the number of objects in the dataset.

The output of this code is a dataframe with questions and their respective cost fuction sorted in the ascending order. That order of questions will be used to form the search tree.

In [ ]:
df = df_amended_for_tangles

questions_with_cost_df = pd.DataFrame(columns=['Question', 'Cost'])
for columnName in df.iteritems():
  qA = df.loc[df[str(columnName[0])] == "1"]
  qB = df.loc[df[str(columnName[0])] == "0"]

  qA = qA.reset_index(drop=True)
  qB = qB.reset_index(drop=True)


  sum_of_similarities = 0

  for i, row in qA.iterrows():
      for l, row in qB.iterrows():
        for k in range(df.shape[1]):
          if qB.iat[l,k] == qA.iat[i,k]: 
            sum_of_similarities = sum_of_similarities +1


  number_of_pairs_between_sets = qA.shape[0]*qB.shape[0]
  if number_of_pairs_between_sets == 0:
    number_of_pairs_between_sets = 0.0000001
  normalized_func_cost = sum_of_similarities/number_of_pairs_between_sets

  questions_with_cost_df = pd.concat([questions_with_cost_df, pd.DataFrame({'Question': [str(columnName[0])],
                    'Cost' : [normalized_func_cost]})], ignore_index = True, axis = 0)

  sum_of_similarities = 0

questions_with_cost_df = questions_with_cost_df[questions_with_cost_df["Question"] != object_identifier_column_name]
questions_with_cost_df = questions_with_cost_df.sort_values(by=['Cost'])
questions_with_cost_df = questions_with_cost_df.reset_index(drop=True)
questions_with_cost_df

This part of the code defines functions used to concat the string with object names for each tree leaf, as well as the function to add children to a specific leaf in the tree.

It also creates the root of the tangle search tree. The root contains the list of all the items in the dataset that are yet to be divided based on the questions/columns.

In [ ]:
def list_of_people_to_string(df):
    list_of_people = ""
    for row in df[object_identifier_column_name]:
      list_of_people = list_of_people + str(row) + ", "
    return list_of_people

def giveChildrenByRootId(root_of_the_tree, parent_name):
  parent=find_by_attr(root_of_the_tree, parent_name)
  child_list=[]

  questions_with_cost_df1 = parent.questions
  if parent.level <= (len(questions_with_cost_df1)):
      people_list = parent.people_list
      
      childlevel=parent.level+1

      column_applicable =  questions_with_cost_df1["Question"].iat[childlevel]

      left = people_list.loc[people_list[str(column_applicable)] == "1"]
      right = people_list.loc[people_list[str(column_applicable)] == "0"]


      if len(left) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_left = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(left)
        childleft = Node(id = id_left, name = id_left, people_list=left, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childleft)

      if len(right) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_right = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(right)
        childright = Node(id=id_right, name=id_right, people_list=right, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childright)
    
 # for pre, fill, node in RenderTree(root_of_the_tree):
  #  print("%s%s" % (pre, node.name))
    
  return "done!"


root = Node(name=list_of_people_to_string(df), people_list=df, questions=questions_with_cost_df, children=[], level=-1)

for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))


This part of the code creates the tree based on answers for each tree object using the list of questions/columns and their cost functions.

It renders the tree using anytree renderer.

In [ ]:
list_of_nodes_checked = []
list_of_nodes_to_check = [node.name for node in PreOrderIter(root)]

while len([node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]) != 0:
  list_of_nodes_to_check = [node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]
  try:
    giveChildrenByRootId(root, list_of_nodes_to_check[0])
  except Exception:
    pass
  list_of_nodes_checked.append(list_of_nodes_to_check[0])


for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))


#Attachment styles in dogs and their relationship with separation-related disorder - tangles example
Original clustering paper: https://www.sciencedirect.com/science/article/abs/pii/S0168159118304921

Dataset: https://data.mendeley.com/datasets/xncmdh494n/1 

##Imports
This implementation of tangle search tree uses anytree python library.


In [1]:
pip install openpyxl==3.0.0

     |████████████████████████████████| 172 kB 7.1 MB/s 
  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241207 sha256=529c235b24c652bbf3f90797932aee531d4b89e15dac82a4e0c0e91f70fd4133
  Stored in directory: /root/.cache/pip/wheels/c7/64/ff/ce98f6e1d2701ae8e216c875da62feed2839ac8a3cae0ab8af
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.9
    Uninstalling openpyxl-3.0.9:
      Successfully uninstalled openpyxl-3.0.9


In [2]:
pip install xlrd==1.2.0

     |████████████████████████████████| 103 kB 8.3 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
pip install anytree

     |████████████████████████████████| 41 kB 458 kB/s 


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np
import openpyxl
from anytree import AnyNode, Node, RenderTree, find, PostOrderIter, PreOrderIter, find_by_attr

##Data preparation

In [14]:
#df_original = pd.read_excel("https://github.com/zuzannazak/Tangles/blob/main/Data%20for_%20Attachment%20styles%20in%20dogs%20and%20their%20relationship%20with%20separation-related%20disord.xlsx?raw=true")
df_original = pd.read_csv('https://raw.githubusercontent.com/zuzannazak/Tangles/main/Data%20for_%20Attachment%20styles%20in%20dogs%20and%20their%20relationship%20with%20separation-related%20disord.csv')
df_original = df_original.reset_index() #optional
df_original.head()


,index,name,breed,mixed,sex,neut,age,SRA,how_long,how_often,where_num,other pet,o_age,origin_num,kept_num,how_long_r,how_often_r,relationship,feelings_home,feelings_known,feelings_strange,deal_home,deal_known,deal_strange,safe_strange,safe_known,safe_home,greet_short,greet_long,whine,bark,howl,drool,pee,poo,destruct,tremble,aroused,owner_feeling,home_behav,elimin_greet,Clusters
0,0,Zara,Vizsla,0,2,1,19,1,2,3,3,2,28,5,3,2,3,7,1,4,5,1,5,5,0,1,3,5,4,2,3,1,1,0,0,1,0,2,0.901895,0.250901,0.748017,1
1,1,nina,Mixed,1,2,0,99,0,2,4,1,0,32,5,3,2,4,7,2,2,3,3,3,5,1,1,3,5,5,1,2,0,0,0,0,0,0,0,-0.064910,0.095491,-1.007548,1
2,2,Tölgy,Mixed,1,1,1,22,1,3,3,1,2,23,4,3,3,3,7,1,2,3,2,3,5,1,1,1,5,5,1,0,0,0,0,0,1,0,0,0.029863,-0.071136,-1.048121,1
3,3,Peggy,Beagle,0,2,1,72,1,2,3,1,0,21,2,2,2,3,6,1,2,4,1,2,4,1,1,3,4,5,1,0,0,0,0,0,0,0,1,-0.206888,-0.842569,-0.344481,1
4,4,Tessa,Mixed,1,2,1,78,1,3,3,1,0,28,1,3,3,3,7,2,2,5,3,3,5,0,1,3,4,5,1,1,0,0,0,0,2,1,2,0.523076,0.328139,-0.355386,1


In [15]:
df_original = df_original.drop(['name', 'mixed'], axis=1)

In [16]:
pd.to_numeric(df_original['breed'], errors='coerce').notnull().all()

False

In [17]:
object_identifier_column_name = 'index' #replace index with the differentiator column if index step was omitted earlier

df_amended_for_tangles = pd.DataFrame()
df_amended_for_tangles[object_identifier_column_name]=df_original[object_identifier_column_name] 

for columnName in df_original.iteritems():
  if columnName[0] == object_identifier_column_name:
    print("this is the ID column")
  else:
    print(columnName[0])
    i=0
    original_column = columnName[0]
    arr = df_original[str(original_column)].to_numpy()
    unique_arr = np.unique(arr) #????????????? dlaczego '<' not supported between instances of 'int' and 'str'?
    while i < len(unique_arr):
      thiscolumnname = original_column + " - " + str(unique_arr[i])
      df_amended_for_tangles[thiscolumnname] = (np.where(df_original[original_column].values == i, "Yes", "No"))
      i=i+1

this is the ID column
breed
sex
neut
age
SRA
how_long
how_often
where_num
other pet
o_age
origin_num
kept_num
how_long_r
how_often_r
relationship
feelings_home
feelings_known
feelings_strange
deal_home
deal_known
deal_strange
safe_strange
safe_known
safe_home
greet_short
greet_long
whine
bark
howl
drool
pee
poo
destruct
tremble
aroused
owner_feeling


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


home_behav
elimin_greet
Clusters


In [13]:
np.unique([0,1,2,'a',2,3,4,3,0,4])

array(['0', '1', '2', '3', '4', 'a'], dtype='<U21')

In [ ]:
a= np.unique(arr)

arr = df.astype({"Column 1": float, "Column 2": int})

TypeError: ignored

In [ ]:
df_amended_for_tangles = df_amended_for_tangles.sample(frac=0.05, replace=False, random_state=1)
df_amended_for_tangles

,index,Reason for absence - 0,Reason for absence - 1,Reason for absence - 2,Reason for absence - 3,Reason for absence - 4,Reason for absence - 5,Reason for absence - 6,Reason for absence - 7,Reason for absence - 8,Reason for absence - 9,Reason for absence - 10,Reason for absence - 11,Reason for absence - 12,Reason for absence - 13,Reason for absence - 14,Reason for absence - 15,Reason for absence - 16,Reason for absence - 17,Reason for absence - 18,Reason for absence - 19,Reason for absence - 21,Reason for absence - 22,Reason for absence - 23,Reason for absence - 24,Reason for absence - 25,Reason for absence - 26,Reason for absence - 27,Reason for absence - 28,Month of absence - 0,Month of absence - 1,Month of absence - 2,Month of absence - 3,Month of absence - 4,Month of absence - 5,Month of absence - 6,Month of absence - 7,Month of absence - 8,Month of absence - 9,Month of absence - 10,...,Height - 178,Height - 182,Height - 185,Height - 196,Body mass index - 19,Body mass index - 21,Body mass index - 22,Body mass index - 23,Body mass index - 24,Body mass index - 25,Body mass index - 27,Body mass index - 28,Body mass index - 29,Body mass index - 30,Body mass index - 31,Body mass index - 32,Body mass index - 33,Body mass index - 34,Body mass index - 35,Body mass index - 36,Body mass index - 38,Absenteeism time in hours - 0,Absenteeism time in hours - 1,Absenteeism time in hours - 2,Absenteeism time in hours - 3,Absenteeism time in hours - 4,Absenteeism time in hours - 5,Absenteeism time in hours - 7,Absenteeism time in hours - 8,Absenteeism time in hours - 16,Absenteeism time in hours - 24,Absenteeism time in hours - 32,Absenteeism time in hours - 40,Absenteeism time in hours - 48,Absenteeism time in hours - 56,Absenteeism time in hours - 64,Absenteeism time in hours - 80,Absenteeism time in hours - 104,Absenteeism time in hours - 112,Absenteeism time in hours - 120
681,681,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
257,257,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
527,527,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
637,637,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
429,429,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No
350,350,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
412,412,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No
189,189,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
670,670,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,N

#Tangles example - Absenteeism at work Data Set
From https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

##Imports and data preparation


In [ ]:
pip install openpyxl==3.0.0

In [ ]:
pip install xlrd==1.2.0

In [ ]:
pip install anytree

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np
import openpyxl
from anytree import AnyNode, Node, RenderTree, find, PostOrderIter, PreOrderIter, find_by_attr

In [ ]:
df_original = pd.read_excel("https://github.com/zuzannazak/Tangles/blob/main/Data_Absenteeism_at_work/Absenteeism_at_work.xls?raw=true")
df_original = df_original.reset_index()
df_original.head()


,index,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,0,11,26,7,3,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,4
1,1,36,0,7,3,1,118,13,18,50,239554,97,1,1,1,1,0,0,98,178,31,0
2,2,3,23,7,4,1,179,51,18,38,239554,97,0,1,0,1,0,0,89,170,31,2
3,3,7,7,7,5,1,279,5,14,39,239554,97,0,1,2,1,1,0,68,168,24,4
4,4,11,23,7,5,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,2


In [ ]:
df_original = df_original.drop(['ID', 'Distance from Residence to Work', 'Transportation expense', 'Work load Average/day '], axis=1)

In [ ]:
df_amended_for_tangles = pd.DataFrame()
df_amended_for_tangles["index"]=df_original["index"]

for columnName in df_original.iteritems():
  if columnName[0] == "index":
    print("this is the ID column")
  else:
    i=0
    original_column = columnName[0]
    arr = df_original[str(original_column)].to_numpy()
    unique_arr = np.unique(arr)
    while i < len(unique_arr):
      thiscolumnname = original_column + " - " + str(unique_arr[i])
      df_amended_for_tangles[thiscolumnname] = (np.where(df_original[original_column].values == i, "Yes", "No"))
      i=i+1


df_amended_for_tangles = df_amended_for_tangles.sample(frac=0.05, replace=False, random_state=1)
df_amended_for_tangles

this is the ID column


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


,index,Reason for absence - 0,Reason for absence - 1,Reason for absence - 2,Reason for absence - 3,Reason for absence - 4,Reason for absence - 5,Reason for absence - 6,Reason for absence - 7,Reason for absence - 8,Reason for absence - 9,Reason for absence - 10,Reason for absence - 11,Reason for absence - 12,Reason for absence - 13,Reason for absence - 14,Reason for absence - 15,Reason for absence - 16,Reason for absence - 17,Reason for absence - 18,Reason for absence - 19,Reason for absence - 21,Reason for absence - 22,Reason for absence - 23,Reason for absence - 24,Reason for absence - 25,Reason for absence - 26,Reason for absence - 27,Reason for absence - 28,Month of absence - 0,Month of absence - 1,Month of absence - 2,Month of absence - 3,Month of absence - 4,Month of absence - 5,Month of absence - 6,Month of absence - 7,Month of absence - 8,Month of absence - 9,Month of absence - 10,...,Height - 178,Height - 182,Height - 185,Height - 196,Body mass index - 19,Body mass index - 21,Body mass index - 22,Body mass index - 23,Body mass index - 24,Body mass index - 25,Body mass index - 27,Body mass index - 28,Body mass index - 29,Body mass index - 30,Body mass index - 31,Body mass index - 32,Body mass index - 33,Body mass index - 34,Body mass index - 35,Body mass index - 36,Body mass index - 38,Absenteeism time in hours - 0,Absenteeism time in hours - 1,Absenteeism time in hours - 2,Absenteeism time in hours - 3,Absenteeism time in hours - 4,Absenteeism time in hours - 5,Absenteeism time in hours - 7,Absenteeism time in hours - 8,Absenteeism time in hours - 16,Absenteeism time in hours - 24,Absenteeism time in hours - 32,Absenteeism time in hours - 40,Absenteeism time in hours - 48,Absenteeism time in hours - 56,Absenteeism time in hours - 64,Absenteeism time in hours - 80,Absenteeism time in hours - 104,Absenteeism time in hours - 112,Absenteeism time in hours - 120
681,681,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
257,257,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
527,527,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
637,637,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No
429,429,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No
350,350,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
412,412,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No
189,189,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
670,670,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,N

##Tangles


In [ ]:
df = df_amended_for_tangles
object_identifier_column_name = "index"


questions_with_cost_df = pd.DataFrame(columns=['Question', 'Cost'])
for columnName in df.iteritems():
  qA = df.loc[df[str(columnName[0])] == "Yes"]
  qB = df.loc[df[str(columnName[0])] == "No"]

  qA = qA.reset_index(drop=True)
  qB = qB.reset_index(drop=True)


  sum_of_similarities = 0

  for i, row in qA.iterrows():
      for l, row in qB.iterrows():
        for k in range(df.shape[1]):
          if qB.iat[l,k] == qA.iat[i,k]: 
            sum_of_similarities = sum_of_similarities +1


  number_of_pairs_between_sets = qA.shape[0]*qB.shape[0]
  if number_of_pairs_between_sets == 0:
    number_of_pairs_between_sets = 0.0000001
  normalized_func_cost = sum_of_similarities/number_of_pairs_between_sets

  questions_with_cost_df = pd.concat([questions_with_cost_df, pd.DataFrame({'Question': [str(columnName[0])],
                    'Cost' : [normalized_func_cost]})], ignore_index = True, axis = 0)

  sum_of_similarities = 0

questions_with_cost_df = questions_with_cost_df[questions_with_cost_df["Question"] != object_identifier_column_name]
questions_with_cost_df = questions_with_cost_df.sort_values(by=['Cost'])
questions_with_cost_df = questions_with_cost_df.reset_index(drop=True)
questions_with_cost_df

In [ ]:
questions_with_cost_df = questions_with_cost_df.drop(questions_with_cost_df[questions_with_cost_df.Cost = 0].index)

In [ ]:
def list_of_people_to_string(df):
    list_of_people = ""
    for row in df[object_identifier_column_name]:
      list_of_people = list_of_people + str(row) + ", "
    return list_of_people

In [ ]:
def giveChildrenByRootId(root_of_the_tree, parent_name):
  parent=find_by_attr(root_of_the_tree, parent_name)
  child_list=[]

  questions_with_cost_df1 = parent.questions
  if parent.level <= (len(questions_with_cost_df1)):
      people_list = parent.people_list
      
      childlevel=parent.level+1

      column_applicable =  questions_with_cost_df1["Question"].iat[childlevel]

      left = people_list.loc[people_list[str(column_applicable)] == "Yes"]
      right = people_list.loc[people_list[str(column_applicable)] == "No"]


      if len(left) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_left = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(left)
        childleft = Node(id = id_left, name = id_left, people_list=left, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childleft)

      if len(right) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_right = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(right)
        childright = Node(id=id_right, name=id_right, people_list=right, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childright)
    
 # for pre, fill, node in RenderTree(root_of_the_tree):
  #  print("%s%s" % (pre, node.name))
    
  return "done!"

In [ ]:
root = Node(name=list_of_people_to_string(df), people_list=df, questions=questions_with_cost_df, children=[], level=-1)

for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 


In [ ]:
list_of_nodes_checked = []
list_of_nodes_to_check = [node.name for node in PreOrderIter(root)]

while len([node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]) != 0:
  list_of_nodes_to_check = [node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]
  try:
    giveChildrenByRootId(root, list_of_nodes_to_check[0])
  except Exception:
    pass
  list_of_nodes_checked.append(list_of_nodes_to_check[0])

In [ ]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 
└── Hit target - 97: 681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 
    └── Weight - 68: 681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 
        └── Weight - 69: 681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 
            └── Weight - 70: 681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, 
    

['Weight - 79: 681, 257, 527, 637, 429, 350, 412, 189, 670, 411, 459, 104, 597, 707, 311, 8, 305, 518, 624, 719, 656, 47, 678, 19, 368, 675, 512, 181, 408, 704, 363, 333, 432, 69, 60, 366, 399, ']

##k-means
https://realpython.com/k-means-clustering-python/

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

df_for_kmeans = df_original

X, _ = make_blobs(n_samples=100, centers=df_for_kmeans.shape[1], n_features=df_for_kmeans.shape[1])

kmeans = KMeans(n_clusters=8)

y = kmeans.fit_predict(df_for_kmeans)

df_for_kmeans['Cluster'] = y

df_for_kmeans.head()

,index,Reason for absence,Month of absence,Day of the week,Seasons,Service time,Age,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,Cluster
0,0,26,7,3,1,13,33,97,0,1,2,1,0,1,90,172,30,4,3
1,1,0,7,3,1,18,50,97,1,1,1,1,0,0,98,178,31,0,3
2,2,23,7,4,1,18,38,97,0,1,0,1,0,0,89,170,31,2,3
3,3,7,7,5,1,14,39,97,0,1,2,1,1,0,68,168,24,4,3
4,4,23,7,5,1,13,33,97,0,1,2,1,0,1,90,172,30,2,3


#Tangles example - fake yes/no questionnaire

In [ ]:
pip install openpyxl==3.0.0

In [ ]:
pip install anytree

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np
import openpyxl
from anytree import AnyNode, Node, RenderTree, find, PostOrderIter, PreOrderIter, find_by_attr


In [ ]:
#df = pd.read_excel("/content/drive/MyDrive/STUDIA/Doktorskie/Tangle/Tangles_example_fake_data.xlsx")
df = pd.read_excel("https://github.com/zuzannazak/Tangles/blob/main/Tangles_example_fake_data.xlsx?raw=true")
df.head()

,Person,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10
0,A,Yes,Yes,No,No,No,No,Yes,No,No,Yes
1,B,Yes,Yes,No,Yes,Yes,No,No,Yes,Yes,Yes
2,C,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes
3,D,Yes,Yes,No,Yes,No,No,No,No,Yes,Yes
4,E,Yes,Yes,No,No,Yes,No,No,Yes,Yes,Yes


In [ ]:
#calculating cost function for cuts - the mean over the similarities over all possible pairs of separated persons:

questions_with_cost_df = pd.DataFrame(columns=['Question', 'Cost'])
for columnName in df.iteritems():
  qA = df.loc[df[str(columnName[0])] == "Yes"]
  qB = df.loc[df[str(columnName[0])] == "No"]

  qA = qA.reset_index(drop=True)
  qB = qB.reset_index(drop=True)


  sum_of_similarities = 0

  for i, row in qA.iterrows():
      for l, row in qB.iterrows():
        for k in range(10):
          if qB.iat[l,k] == qA.iat[i,k]: 
            sum_of_similarities = sum_of_similarities +1
            #print(str(j) + ": rząd " + str(l) + ", pytanie " + str(k) + ": " + q1B.iat[l,k] + " i "+"rząd " + str(i) +", pytanie " + str(k) + ": " + q1A.iat[i,k])
            #print(sum_of_similarities)

  number_of_pairs_between_sets = qA.shape[0]*qB.shape[0]
  if number_of_pairs_between_sets == 0:
    number_of_pairs_between_sets = 0.0000001
  normalized_func_cost = sum_of_similarities/number_of_pairs_between_sets

  questions_with_cost_df = pd.concat([questions_with_cost_df, pd.DataFrame({'Question': [str(columnName[0])],
                    'Cost' : [normalized_func_cost]})], ignore_index = True, axis = 0)

  sum_of_similarities = 0

questions_with_cost_df = questions_with_cost_df[questions_with_cost_df["Question"] != "Person"]
questions_with_cost_df = questions_with_cost_df.sort_values(by=['Cost'])
questions_with_cost_df = questions_with_cost_df.reset_index(drop=True)
questions_with_cost_df
#why 0 when all of them are different, or all of them are the same?

,Question,Cost
0,Question 2,0.000000
1,Question 3,4.404762
2,Question 6,4.531746
3,Question 1,4.641667
4,Question 9,4.727273
5,Question 8,4.761905
6,Question 5,4.769841
7,Question 7,4.785714
8,Question 4,4.823077
9,Question 10,5.225000


In [ ]:
def list_of_people_to_string(df):
    list_of_people = ""
    for row in df["Person"]:
      list_of_people = list_of_people + str(row) + ", "
    return list_of_people

In [ ]:
def giveChildrenByRootId(root_of_the_tree, parent_name):
  parent=find_by_attr(root_of_the_tree, parent_name)
  child_list=[]

  questions_with_cost_df1 = parent.questions
  if parent.level <= (len(questions_with_cost_df1)):
      people_list = parent.people_list
      column_applicable =  questions_with_cost_df1["Question"].iat[parent.level]

      left = people_list.loc[people_list[str(column_applicable)] == "Yes"]
      right = people_list.loc[people_list[str(column_applicable)] == "No"]

      childlevel=parent.level+1

      if len(left) != 0 and len(people_list)>1 and parent.level<len(questions_with_cost_df1)-1:
        id_left = questions_with_cost_df1["Question"].iat[parent.level] + ": " + list_of_people_to_string(left)
        childleft = Node(id = id_left, name = id_left, people_list=left, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childleft)

      if len(right) != 0 and len(people_list)>1 and parent.level<len(questions_with_cost_df1)-1:
        id_right = questions_with_cost_df1["Question"].iat[parent.level] + ": " + list_of_people_to_string(right)
        childright = Node(id=id_right, name=id_right, people_list=right, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childright)
    
 # for pre, fill, node in RenderTree(root_of_the_tree):
  #  print("%s%s" % (pre, node.name))
    
  return "done!"

In [ ]:
def giveChildrenByRootId(root_of_the_tree, parent_name):
  parent=find_by_attr(root_of_the_tree, parent_name)
  child_list=[]

  questions_with_cost_df1 = parent.questions
  if parent.level <= (len(questions_with_cost_df1)):
      people_list = parent.people_list
      
      childlevel=parent.level+1

      column_applicable =  questions_with_cost_df1["Question"].iat[childlevel]

      left = people_list.loc[people_list[str(column_applicable)] == "Yes"]
      right = people_list.loc[people_list[str(column_applicable)] == "No"]


      if len(left) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_left = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(left)
        childleft = Node(id = id_left, name = id_left, people_list=left, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childleft)

      if len(right) != 0 and len(people_list)>1 and childlevel<len(questions_with_cost_df1):
        id_right = questions_with_cost_df1["Question"].iat[childlevel] + ": " + list_of_people_to_string(right)
        childright = Node(id=id_right, name=id_right, people_list=right, questions=questions_with_cost_df1, level = childlevel, parent=find_by_attr(root_of_the_tree, parent_name))
        child_list.append(childright)
    
 # for pre, fill, node in RenderTree(root_of_the_tree):
  #  print("%s%s" % (pre, node.name))
    
  return "done!"

In [ ]:
root = Node(name=list_of_people_to_string(df), people_list=df, questions=questions_with_cost_df, children=[], level=-1)

for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, W, Y, Z, 


In [ ]:
list_of_nodes_checked = []
list_of_nodes_to_check = [node.name for node in PreOrderIter(root)]

while len([node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]) != 0:
  list_of_nodes_to_check = [node.name for node in PreOrderIter(root, filter_=lambda n: n.name not in list_of_nodes_checked)]
  try:
    giveChildrenByRootId(root, list_of_nodes_to_check[0])
  except Exception:
    pass
  list_of_nodes_checked.append(list_of_nodes_to_check[0])

  



In [ ]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, W, Y, Z, 
└── Question 2: A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, W, Y, Z, 
    ├── Question 3: W, Z, 
    │   ├── Question 6: W, 
    │   └── Question 6: Z, 
    └── Question 3: A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U, Y, 
        ├── Question 6: F, G, H, I, J, N, P, S, 
        │   ├── Question 1: S, 
        │   └── Question 1: F, G, H, I, J, N, P, 
        │       ├── Question 9: G, I, J, N, 
        │       │   ├── Question 8: G, J, N, 
        │       │   │   └── Question 5: G, J, N, 
        │       │   │       └── Question 7: G, J, N, 
        │       │   │           ├── Question 4: J, 
        │       │   │           └── Question 4: G, N, 
        │       │   │               └── Question 10: G, N, 
        │       │   └── Question 8: I, 
        │       └── Question 9: F, H, P, 
        │           ├── Question 8: H, P, 
        │           │   └── Question 5: H, P, 
        │   

In [ ]:
#dane klasyfikacyjne i spróbować na nich stworzyć drzewo (k-średnich itp), porównać wyniki

##Naive table search algorythm
**Input:** The separation system S, enumerated as s1, . . . , sn; oracle access to F.

**Output:** For every i ≤ n a list Ti of all F-tangles of {s1, . . . , si}.